In [26]:
import os
import sys
import re
import pandas as pd
import numpy as np
from enum import Enum
from typing import Optional, Union, List, Callable
from dataclasses import dataclass

PKG_ROOT = os.path.dirname(os.path.realpath(os.getcwd()))
if not PKG_ROOT in sys.path:
    sys.path.append(PKG_ROOT)

In [27]:
@dataclass
class SegBase:
    
    buffer: bytearray
    pos: int
    length: int
    signed: bool=False
    order: str='little'
    skiphead: int=0
    skiptail: int=0
    
    @staticmethod
    def bytes2Num(barray, signed:bool=False, order:str='little'):
        return int.from_bytes(barray, signed=signed, byteorder=order)
    
    @staticmethod
    def num2Bytes(num, length, signed=False, order='little'):
        return num.to_bytes(length, signed=signed, byteorder=order)
    
    def get_bytes(self):
        return self.buffer[self.pos+self.skiphead
                          :self.pos+self.skiphead-self.skiptail]
        
    def get_value(self):
        return self.bytes2Num(self.get_bytes(), self.signed, self.order)
    
    def get_string(self):
        s = self.get_bytes()
        p = np.argmax(np.logical_and(np.array(s) > 127
                                     ,np.array(s) < 32))
        return str(s[:p], encoding='latin')
    
    def set_value(self, value:Union[int, Enum]):
        if hasattr(value, 'value'):
            value = value.value
        self.buffer[self.pos+self.skiphead
                   :self.pos+self.skiphead-self.skiptail] = self.num2Bytes(value, self.length, self.signed, self.order) 
            
    def inc_value(self, inc_value:int=1):
        self.set_value(self.get_value() + inc_value)
    